elem 환경에서 실행

In [ ]:
#cpu 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

train_df.head(10)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre..."


In [3]:
TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [5]:
def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

In [6]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

In [7]:
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

In [8]:
#특수문자 제거 및 전처리 코드
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    return text

In [9]:
sampled_df['text']=sampled_df['text'].apply(clean_text)

In [10]:
sampled_df = sampled_df.reset_index(drop=True)

In [11]:
train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

In [12]:
x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

In [13]:
to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [14]:
len(to_txt_filter)

20000

엘모벡터 생성

In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 
# Location of pretrained LM.  Here we use the test fixtures.
datadir = os.path.join('C:/Users/user/Desktop/bilm-tf-master/swb', 'model(ag256)') #사전 학습된 모델이 있는 디렉토리
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
# Create a Batcher to map text to character ids.
batcher = Batcher(vocab_file, 50)
 
# Input placeholders to the biLM.
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
# Build the biLM graph.
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
# Get ops to compute the LM embeddings.
context_embeddings_op = bilm(context_character_ids)
 
# Get an op to compute ELMo (weighted average of the internal biLM layers)
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)
 
# Now we can compute embeddings.
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [17]:
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
 
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [18]:
# Computing euclidean distance between words embedding
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
# Computing cosine distance between words embedding
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  12.98 
Dress-Fashion =  12.22


Cosine Distance Comparison - 

Dress-Technology =  0.8 
Dress-Fashion =  0.71


임베딩

In [19]:
except_lst=[]

In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(16000):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/train(ag256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 74, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (

...

Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  

Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (

In [21]:
#12935번째 문서는 길이가 너무 커서 GPU가 아닌 CPU로 실행하여 임베딩 벡터를 저장할 것, 첫번째 코드를 실행해서 CPU로 바꿔 실행 가능
except_lst

[]

In [19]:
except_test_lst=[]

In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(4000):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/test(ag256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (

...

Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 44, 256)


In [21]:
except_test_lst

[]